We want to count ordered pairs of functions $(f,g)$ that satisfy certain identities.  This notebook collects identities where there is a straightforward combinatorial counting argument for such pairs.

First, let us look at $gf=f$, ([A239750](https://oeis.org/A239750)) which has been completely solved previously, but nicely illustrates the basic combinatorial reasoning for these problems.  Here $g$ is the identity on the image of $f$, and arbitrary elsewhere.  There are two ways of looking at this, first if $k$ is the size of the image of $f$ then there are $k!\left\{n\atop k\right\}$ such functions $f$ and $n^{n-k}$ functions $g$ that are the identity on the image of $f$, so we have
$$\#(f,g)=\sum_{k=0}^n\binom{n}{k}\left\{n\atop k\right\}k!\; n^{n-k}.$$

On the other hand, if we start with functions $g$ that are the identity on $k$ points, then $f$ must map into the points where $g$ is the identity.  There are ${(n-1)}^{n-k}$ such functions $g$, and $k^n$ such functions $f$ so we also have
$$\#(f,g)=\sum_{k=0}^n\binom{n}{k}k^n{(n-1)}^{n-k}$$

Note that this also counts $gf=f^r$ for any $r>0$ as $g$ is completely specified on the image of $f$ and arbitrary elsewhere, exactly the same conditions we had for $r=2$.

In [3]:
def a239750_1(n):
    return sum(binomial(n,k) * stirling_number2(n,k) * factorial(k) * n^(n-k) for k in srange(n+1))

def a239750_2(n):
    return sum(binomial(n,k) * k^n * (n-1)^(n-k) for k in srange(n+1))

print all(a239750_1(n) == a239750_2(n) for n in range(20))
print [a239750_2(n) for n in range(10)]

True
[1, 1, 6, 87, 2200, 84245, 4492656, 315937195, 28186856832, 3099006365769]


Next, let us look at $gff=f$ ([A235328](https://oeis.org/A235328)).  Here, $f=g^{-1}$ on the image of $f$, in particular both $f$ and $g$ are invertible on the image of $f$.  If there are $k$ points in the image of $f$ then there are $k!$ ways of assigning $f$ on the image of $f$, which then completely specifies $g$ on the image of $f$.  On the points outside the image of $f$ there are $k^{n-k}$ ways of assigning $f$ since it must map into its image.  Since $f$ permutes the elements of the image of $f$, we know that every point in the image of $f$  is mapped by $f$ from some point in the image of $f$, so we do not need to worry about only mapping to a subset.  Finally $g$ is completely arbitrary outside the image of $f$ so there are $n^{n-k}$ ways of assigning it.  So combining everything we have
$$\#(f,g)=\sum_{k=0}^n\binom{n}{k}k!\;(nk)^{n-k}.$$

The same reasoning holds for $gf^r=f$ where $r>1$, as both $f$ and $g$ are still invertible on the image of $f$, and $g$ is arbitrary off the image of $f$.

In [19]:
def a235328(n):
    return sum(falling_factorial(n,k) * (n*k)^(n-k) for k in srange(n+1))

import itertools as it
def count_coincidences(n, test):
    count = 0
    for f in it.product(range(n), repeat=n):
        for g in it.product(range(n), repeat=n):
            good = test(n,f,g)
            if good:
                count += 1
    return count

def check_cube(n, f, g):
    for i in range(n):
        if f[i] != g[f[f[f[i]]]]:
            return False
    return True


print [count_coincidences(n, check_cube) for n in range(0,6)]
print [a235328(n) for n in range(10)]

[1, 1, 6, 69, 1336, 39145]
[1, 1, 6, 69, 1336, 39145, 1598256, 85996561, 5872177536, 494848403793]


Let us look at $ggf=f$ ([A239771](https://oeis.org/A239771)).  Here $g$ is an involution on the image of $f$.  

We can divide the image of $f$ up into three parts, points $x$ where $g(x)=x$, pairs of points $(x,y)$ with both $x$ and $y$ in the image of $f$ where $g(x)=y$ and $g(y)=x$, and points $x$ where $g(x)$ is not in the image of $f$ and $g(g(x))=x$.

This gives us

$$\#(f,g)=\sum_{k=0}^n \binom{n}{k}k!\left\{n \atop k\right\} \sum_{j=0}^{\min(k,n-k)}\binom{k}{j}\frac{(n-k)!n^{n-k-j}}{(n-k-j)!} \sum_{i=0}^{\lfloor\frac{k-j}{2}\rfloor}\frac{(k-j)!}{(k-j-2i)!i!2^i}
$$

The inner sum is [A000085](https://oeis.org/A000085) and satisfies the nice recurrence $a_n = a_{n-1}+(n-1)a_{n-2}$. 

In [17]:
def a239771(n):
    return sum(falling_factorial(n,k) * stirling_number2(n,k) *
              sum(binomial(k,j) * falling_factorial(n-k,j) * n^(n-k-j) *
                 sum(falling_factorial(k-j,2 * i)/(factorial(i)*2^i)
                    for i in range(floor((k-j)/2)+1))
                 for j in range(min(k,n-k)+1))
              for k in range(n+1))

@CachedFunction
def involutions(n):
    return 1 if n < 2 else involutions(n-1) + (n-1) * involutions(n-2)

def a239771_b(n):
    return sum(falling_factorial(n,k) * stirling_number2(n, k) *
              sum(binomial(k,j)*falling_factorial(n-k,j)*n^(n-k-j)*
                  involutions(k-j)
                 for j in range(min(k,n-k)+1))
              for k in range(n+1))
print [a239771(n) for n in range(10)]
print [a239771_b(n) for n in range(10)]

[1, 1, 10, 213, 8056, 465945, 37823616, 4075467781, 560230714240, 95369455852497]
[1, 1, 10, 213, 8056, 465945, 37823616, 4075467781, 560230714240, 95369455852497]


This logic works (with increasing difficulty with $m$ for $g^r f=f$).  For $r=3$ we need to count the 3 cycles that have all points, two points, or one point in the image of $f$.

If we let $k$ be the size of the image of $f$, $l$ be the number of three-cycles having one point in the image of $f$, $j$ being the number having two points in the image of $f$ and $i$ being the number of three cycles having three points in the image of $f$.